In [29]:
#def init():
    #CHANGE THIS TO YOUR TRAIN FOLDER
image_folder = 'C:/Study/Semester2/Machine Learning/ML_images/Training' 
#image_folder = 'C:/Users/danie/Desktop/Training_ML'

TOO_SMALL = 200000
SMALL = 500000
MEDIUM = 1200000
LARGE = 2100000
    
X_train = []
Y_train = []
counter = 1
    
img_height = 1888 #Usually 128. this might change depending on Annaya & Danielle's input 
img_width = 1888 #Usually 128. this might change depending on Annaya & Danielle's input 
epochs = 10 #Start with 1, and increase to 10, 100, 500, 1000 and 3000. 11 is the recommended number of runs through the training dataset. We will probably have to tune this. 
#total_training = total number of training data len()
batch_size = 1 #test different ones from 1 to 2, 5, 10, etc
total_images = 1

In [30]:
#pip install opencv-python

In [31]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import skimage
import cv2
import glob
import pandas as pd

from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, UpSampling2D
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage import data, io,color
from skimage.io import imread_collection #loads a collection of images
from skimage.io import imread, imshow
from pathlib import Path

from sklearn.decomposition import PCA

import matplotlib as mpl
import matplotlib.pyplot as plt

In [32]:
#Define sizes of images based on their resolution
#Image shape and resolution

#TODO make the IMG sizes make sense


def shape(img):
    size = 0
    resolution = img.shape[0] * img.shape[1]
    if resolution <= TOO_SMALL:
        size = 0
    elif resolution <= SMALL:
        size = 1
    elif resolution <= MEDIUM:
        size = 2
    elif resolution <= LARGE:
        size = 3
    else:
        size = 6
    return size

In [33]:
def create_dataframe(image_folder):
    images = Path(image_folder).glob('*.jpg')

    list_of_files = []
    for image in images:
        list_of_files.append(str(image))

    df = pd. DataFrame(list_of_files, columns = ["image_path"])
    df_length = len(df.index)
    print("processing", df_length, "photos for training")
    
    height= []
    width = []
    resolution = []
    
    for image_path in df['image_path']:
        img = cv2.imread(str(image_path))
        height.append(img.shape[0])
        width.append(img.shape[1])
        
    df["height"] = height
    df["width"] = width
    df["resolution"] = df["height"] * df["width"]
    
    df = df[df.resolution < LARGE]
    
    ll = df_length - len(df.index)
    print("dropped ", ll, "photos because they were too large to process")
    
    df = df[df.resolution > TOO_SMALL]
    
    ss = df_length - len(df.index) - ll
    print("dropped ", ss, "photos because they were too small to process")
    df.head()
    
    return df

In [34]:
#PCA rgb then pass to prep_img as an rgb image

def pca_rgb(imgBGR, size): #imgRGB, size
    b,g,r = cv2.split(imgBGR)
    list_channels = [b,g,r]
    
    #check size first
    if size ==1:
        inverted_img = []
        for c in list_channels:            
            pca = PCA(n_components = 100)
            transformed = pca.fit_transform(c/255) #r_scaled
            inverted = pca.inverse_transform(transformed)
            inverted_img.append(inverted)
        
        tuple(inverted_img)
        RGB_compressed = cv2.merge(inverted_img)    

    elif size == 2:
        inverted_img = []
        for c in list_channels:            
            pca = PCA(n_components = 300)
            transformed = pca.fit_transform(c/255) #r_scaled
            inverted = pca.inverse_transform(transformed)
            inverted_img.append(inverted)
        
        tuple(inverted_img)
        RGB_compressed = cv2.merge(inverted_img)
    else:
        inverted_img = []
        for c in list_channels:            
            pca = PCA(n_components = 500)
            transformed = pca.fit_transform(c/255) #r_scaled
            inverted = pca.inverse_transform(transformed)
            inverted_img.append(inverted)
        
        tuple(inverted_img)
        RGB_compressed = cv2.merge(inverted_img)
        
    RGB_compressed = np.float32(RGB_compressed)
        
    return RGB_compressed

In [35]:
# image padding

def rgb_pad(image):
    
    height = image.shape[0]
    width = image.shape[1]

    WHITE = [255,255,255] #[255,255,255]  1, 1, 1
    MAX = 1888
    
    if height % 2 == 0:
        top = int((MAX - height)/ 2)
        bottom = int((MAX - height)/ 2)
    else: 
        top = int((MAX - (height-1))/ 2)
        bottom = int(((MAX - (height-1))/ 2)-1)

    if width % 2 == 0:
        left = int((MAX - width)/ 2)
        right = int((MAX - width)/ 2)
    else: 
        left = int((MAX - (width-1))/ 2)
        right = int(((MAX - (width-1))/2)-1)
    
    image = cv2.copyMakeBorder(image, top, bottom, left, right,
    cv2.BORDER_CONSTANT,value=WHITE)

    return image

In [58]:
#Debugged function
def prep_img(img_path):
    imgRGB = img_to_array(load_img(img_path))
    imgRGB = np.array(imgRGB, dtype=float)

    
    #check the image category by 'shape' function
    image_size_category = shape(imgRGB) 
    #print("image size determined!")

    if image_size_category <1:
        raise Exception("Your image is too small to be used")
    elif image_size_category >4:
        raise Exception("Your image is too big to be used")

    #print("image PCA started!")
 
    RGB_compressed = pca_rgb(imgRGB, image_size_category) 
    print('RGB_compressed:', np.min(RGB_compressed), np.max(RGB_compressed))
    #plt.figure(figsize=(10, 10))
    #plt.imshow(RGB_compressed)
    
    print("image is padded")
    padded_image = rgb_pad(RGB_compressed)
  
    #plt.figure(figsize=(10, 10))
    #plt.imshow(padded_image) # show after rgb compressed after padded
    
    #image_pca = np.float32((1.0/255)*image_pca) # #The rgb_compressed come out in RGB scaled data already
    print("image converted to LAB")
    padded_LAB = cv2.cvtColor(padded_image, cv2.COLOR_RGB2Lab) #convert padded RGB compressed to LAB
    print('A:', np.min(padded_LAB), np.max(padded_LAB))
    
    #plt.figure(figsize=(10, 10))
    #plt.imshow(padded_image, cmap = plt.cm.gray) #show after convert padded rgb compressed to RGB with the function
    #imgLAB = np.clip(imgLAB, 0, 1)

    
    #separate L and AB color channels
    Y = padded_LAB[:, :,1:]
    Y = Y / 128 #comment out when show predicted
    print('A normalized:', np.min(Y[:,:,0]), np.max(Y[:,:,0]))
    print('B normalized:', np.min(Y[:,:,0]), np.max(Y[:,:,0]))
    
    X = padded_LAB[:, :, 0] 
    print('X:', np.min(X), np.max(X))
    #plt.figure(figsize=(10, 10)) # show converted RGB to L channel 
    #plt.imshow(X)
    
    #this is in sample code and idk why
    
    X = X.reshape(1, 1888, 1888, 1) #comment out when show predicted
    Y = Y.reshape(1, 1888, 1888, 2) #comment out when show predicted
    
    print("image processed")

    return Y, X 

In [56]:
def prep_dataframe(df):
    for index, row in df.iterrows():
        Y, X = prep_img(row["image_path"])
        Y_train.append(Y)
        X_train.append(X)

# Use the code below to process ALL photos and feed them to the model!!!

In [57]:
df = create_dataframe(image_folder)
prep_dataframe(df)

#add padd(image).tolist()
#caused TypeError: unsupported operand type(s) for /: 'list' and 'int'


processing 151 photos for training
dropped  9 photos because they were too large to process
dropped  5 photos because they were too small to process
RGB_compressed: 0.0074463133 0.9853132
image is padded
image converted to LAB
A: -42.5 100.0
A: -0.21948242 0.3572998
B: -0.21948242 0.3572998
X: 0.8605957 100.0
image processed
RGB_compressed: -0.17393492 1.1337924
image is padded
image converted to LAB
A: -47.359375 100.0
A: -0.29858398 0.60058594
B: -0.29858398 0.60058594
X: 0.0 100.0
image processed
RGB_compressed: -0.021102002 1.022073
image is padded
image converted to LAB
A: -49.703125 100.0
A: -0.19250488 0.30078125
B: -0.19250488 0.30078125
X: 0.0 100.0
image processed
RGB_compressed: -0.0751553 1.050764
image is padded
image converted to LAB
A: -28.59375 100.0
A: -0.22338867 0.38256836
B: -0.22338867 0.38256836
X: 0.0 100.0
image processed
RGB_compressed: -0.018501235 1.0222906
image is padded
image converted to LAB
A: -31.25 100.0
A: -0.24414062 0.29248047
B: -0.24414062 0.29248

RGB_compressed: -0.10807131 1.1213304
image is padded
image converted to LAB
A: -52.890625 100.0
A: -0.30895996 0.30322266
B: -0.30895996 0.30322266
X: 0.0 100.0
image processed
RGB_compressed: -0.029292095 1.0299101
image is padded
image converted to LAB
A: -21.5625 100.0
A: -0.16491699 0.39611816
B: -0.16491699 0.39611816
X: 0.0 100.0
image processed
RGB_compressed: -0.070702076 1.0824528
image is padded
image converted to LAB
A: -63.828125 100.0
A: -0.45251465 0.54833984
B: -0.45251465 0.54833984
X: 0.0 100.0
image processed
RGB_compressed: -0.0286018 1.032088
image is padded
image converted to LAB
A: -58.4375 100.0
A: -0.34387207 0.3112793
B: -0.34387207 0.3112793
X: 0.0 100.0
image processed
RGB_compressed: -0.038568106 1.0246221
image is padded
image converted to LAB
A: -27.96875 100.0
A: -0.1418457 0.2902832
B: -0.1418457 0.2902832
X: 0.0 100.0
image processed
RGB_compressed: -0.049552802 1.0552194
image is padded
image converted to LAB
A: -39.28125 100.0
A: -0.30688477 0.395507

A: -38.65625 100.0
A: -0.30200195 0.33728027
B: -0.30200195 0.33728027
X: 0.0 100.0
image processed
RGB_compressed: -0.04936067 1.0432223
image is padded
image converted to LAB
A: -41.5 100.0
A: -0.25024414 0.3701172
B: -0.25024414 0.3701172
X: 0.12207031 100.0
image processed
RGB_compressed: -0.024087714 1.0525559
image is padded
image converted to LAB
A: -100.3125 100.0
A: -0.57299805 0.5821533
B: -0.57299805 0.5821533
X: 0.20141602 100.0
image processed
RGB_compressed: -0.09928114 1.0789912
image is padded
image converted to LAB
A: -64.859375 100.0
A: -0.42980957 0.59350586
B: -0.42980957 0.59350586
X: 0.0 100.0
image processed
RGB_compressed: -0.09920036 1.1028641
image is padded
image converted to LAB
A: -96.3125 100.0
A: -0.6604004 0.70288086
B: -0.6604004 0.70288086
X: 0.4272461 100.0
image processed
RGB_compressed: -0.056639176 1.0446461
image is padded
image converted to LAB
A: -48.96875 100.0
A: -0.3059082 0.41381836
B: -0.3059082 0.41381836
X: 0.0 100.0
image processed
RGB_c

In [42]:
def mode(): 
    model1 = Sequential([
        #encoder
        Conv2D(128, 2, strides = 2, padding = "same", activation = "relu", input_shape= (1888, 1888, 1)), #testing shape changes
        Conv2D(256, 2, strides = 2, padding = "same", activation = "relu"), 
        Conv2D(256, 2, strides = 2, padding = "same", activation = "relu"), #first param to 512

        #decoder
        Conv2D(256, 2, padding = "same", activation = "relu"), 
        UpSampling2D(2),
        Conv2D(128, 2, padding = "same", activation = "relu"), 
        UpSampling2D(2), 
        Conv2D(2, 2, padding = "same", activation = "tanh"),
        UpSampling2D(2),
    ])
    model1.compile(optimizer ="adam", loss = "mse", metrics = ["accuracy"])
    return model1

In [43]:
model1 = mode()
model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 944, 944, 128)     640       
                                                                 
 conv2d_7 (Conv2D)           (None, 472, 472, 256)     131328    
                                                                 
 conv2d_8 (Conv2D)           (None, 236, 236, 256)     262400    
                                                                 
 conv2d_9 (Conv2D)           (None, 236, 236, 256)     262400    
                                                                 
 up_sampling2d_3 (UpSampling  (None, 472, 472, 256)    0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 472, 472, 128)     131200    
                                                      

In [44]:
len(X_train)

137

In [45]:
len(Y_train)

137

In [46]:
train_data = tf.data.Dataset.from_tensor_slices((X_train, Y_train))

In [50]:
epochs = 60
history = model1.fit(train_data, batch_size = batch_size, epochs = epochs, verbose = 2)  # add validation_split = 0.2

Epoch 1/60
137/137 - 460s - loss: 0.0027 - accuracy: 0.5036 - 460s/epoch - 3s/step
Epoch 2/60
137/137 - 448s - loss: 0.0027 - accuracy: 0.4404 - 448s/epoch - 3s/step
Epoch 3/60
137/137 - 449s - loss: 0.0027 - accuracy: 0.4616 - 449s/epoch - 3s/step
Epoch 4/60
137/137 - 448s - loss: 0.0027 - accuracy: 0.4378 - 448s/epoch - 3s/step
Epoch 5/60
137/137 - 446s - loss: 0.0027 - accuracy: 0.4484 - 446s/epoch - 3s/step
Epoch 6/60
137/137 - 447s - loss: 0.0027 - accuracy: 0.4422 - 447s/epoch - 3s/step
Epoch 7/60
137/137 - 446s - loss: 0.0027 - accuracy: 0.4575 - 446s/epoch - 3s/step
Epoch 8/60
137/137 - 451s - loss: 0.0027 - accuracy: 0.4764 - 451s/epoch - 3s/step
Epoch 9/60
137/137 - 447s - loss: 0.0027 - accuracy: 0.4720 - 447s/epoch - 3s/step
Epoch 10/60
137/137 - 445s - loss: 0.0027 - accuracy: 0.5193 - 445s/epoch - 3s/step
Epoch 11/60
137/137 - 446s - loss: 0.0027 - accuracy: 0.4805 - 446s/epoch - 3s/step
Epoch 12/60
137/137 - 446s - loss: 0.0026 - accuracy: 0.4959 - 446s/epoch - 3s/step
E

In [51]:
filename = 'my_model_tf_F60'
model1.save(filename)

INFO:tensorflow:Assets written to: my_model_tf_F60\assets


## Test one image

In [ ]:
#import model

In [ ]:
# load the model from disk
#reconstructed_model = keras.models.load_model("my_model_tf")

In [ ]:
test_path = "C:/Study/Semester2/Machine Learning/ML - Yes Photos/1900s_loose_bw_0036_mom.jpg"

In [ ]:
def prep_img_test(img_path):
    imgRGB = cv2.imread(img_path)
    imgBGR = np.flip(imgRGB, -1)
    
    #check the image category by 'shape' function
    image_size_category = shape(imgBGR) #imgRGB
    #print("image size determined!")

    if image_size_category <1:
        raise Exception("Your image is too small to be used")
    elif image_size_category >4:
        raise Exception("Your image is too big to be used")

    #print("image PCA started!")
    #image_pca = pca_rgb(imgRGB, image_size_category) 
    RGB_compressed = pca_rgb(imgBGR, image_size_category) #imgRGB, image_size_category
    #RGB_compressed[400][400]
    #plt.figure(figsize=(10, 10))
    #plt.imshow(RGB_compressed)
    
    print("image is padded")
    padded_image = rgb_pad(RGB_compressed)
    #padded_image[1000][1000]
    #plt.figure(figsize=(10, 10))
    #plt.imshow(padded_image)
    
    #image_pca = np.float32((1.0/255)*image_pca) # #The rgb_compressed come out in RGB scaled data already
    print("image converted to LAB")
    padded_LAB = cv2.cvtColor(padded_image, cv2.COLOR_RGB2Lab) #convert padded RGB compressed to LAB
    padded_LAB = padded_LAB/np.amax(padded_LAB) #scaled to -1/1
    #imgLAB = np.clip(imgLAB, 0, 1)

    
    #separate L and AB color channels
    Y = padded_LAB[:, :,1:]
    #Y = Y / 128 #comment out when show predicted
    #Y = Y.tolist()
    X = padded_LAB[:, :, 0] #.tolist()
    #print(X[1000])
    #plt.figure(figsize=(10, 10))
    #plt.imshow(X)
    
    #this is in sample code and idk why
    
    X = X.reshape(1, 1888, 1888, 1) #comment out when show predicted
    Y = Y.reshape(1, 1888, 1888, 2) #comment out when show predicted
    
    print("image processed")

    return Y, X 

In [ ]:
CL, BW = prep_img_test(test_path)
#test_data = tf.data.Dataset.from_tensor_slices(BW)
test_predict = model1.predict(BW) #what shape does the prediction takes?

In [ ]:
type(test_predict)

In [ ]:
test_predict.shape # use this to merge

In [ ]:
BW.shape # use this to merge

In [ ]:
CL.shape

#### merge with d stack

In [ ]:
test_predict_reshaped = test_predict[0,:,:,:]

In [ ]:
test_predict_reshaped.shape

In [ ]:
test_predict_reshaped[0][0]

In [ ]:
plt.imshow(test_predict_reshaped[:,:,0])

In [ ]:
plt.imshow(test_predict_reshaped[:,:,1])

In [ ]:
BW_reshaped  = BW[0,:,:,:]

In [ ]:
BW_reshaped.shape

In [ ]:
plt.imshow(BW_reshaped)

In [ ]:
test_merged_LAB = np.dstack((BW_reshaped, test_predict_reshaped))

In [ ]:
test_merged_LAB.shape

In [ ]:
test_merged_LAB[0][0]

In [ ]:
plt.imshow(test_merged_LAB)

In [ ]:
#test_merged_converted = color.Lab2rgb(test_merged)
test_merged_rgb = test_merged_rgb*128

In [ ]:
plt.imshow(test_merged_rgb)

In [ ]:
test_merged_rgb = cv2.cvtColor(test_merged, cv2.COLOR_Lab2RGB) 

In [ ]:
test_merged_rgb[0][0] # this is weird

In [ ]:
#test_merged_rgb_scaled = (test_merged_rgb/255).astype('float32')

In [ ]:
#plt.imshow(test_merged_rgb)
plt.imshow(test_merged_rgb_scaled )

In [ ]:
plt.imsave('result1.jpg', test_merged_rgb)

##### a way to merge the prediction

In [ ]:
#pastes predicted AB to RGB blank canvas
canvas = np.zeros((1888,1888,3))
canvas[:,:,0] = BW[0][:,:,0]
canvas[:,:,1:] =test_predict[0]

In [ ]:
#convert AB (in RGB space) to rgb photo
rgb_converted = color.lab2rgb(canvas)

In [ ]:
plt.imshow(rgb_converted)

In [ ]:
lab_2rgb = cv2.cvtColor(canvas, cv2.COLOR_RGB2Lab)

In [ ]:
lab_to = 

In [ ]:
lab_merge.shape

In [ ]:
plt.imshow(canvas)

In [ ]:
#Test input array

In [ ]:
img_path1 = 'C:/Study/Semester2/Machine Learning/ML - Yes Photos/1900s_loose_bw_0036_mom.jpg'
Y01, X01 = prep_img(img_path1)

In [ ]:
type(X01[0])

In [ ]:
#show AB channel of the image
plt.imshow(X01) #X01[0,:,:,0]

In [ ]:
df_X01 = pd.DataFrame(X01[0,:,:,0])

In [ ]:
df_Y01 = pd.DataFrame(Y01[0,:,:,0])

In [ ]:
df_Y01.describe()